In [18]:
from variables import countries_msft,countries_eubucco

In [2]:
import pandas as pd

In [19]:
out_msft = {country:{} for country in countries_msft}
out_eubucco = {country:{} for country in countries_eubucco}

In [21]:
for country in countries_msft:

    df_msft = pd.read_csv(f'../overviews/overview-msft/{country}_overview.csv')

    if country in countries_eubucco:    
        df_eubucco = pd.read_csv(f'../overviews/overview-v0_1/{country}_overview.csv')
        iterator = ((out_msft,df_msft),(out_eubucco,df_eubucco))
    
    else: iterator = ((out_msft,df_msft))

    for d,df in iterator:
        d[country]['Total Footprint Area'] = round(df.a_tot.sum())
        d[country]['Total Footprint Number'] = round(df.bldgs_n_tot.sum())

/tmp/ipykernel_194690/1466649161.py:3: DtypeWarning: Columns (60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df_msft = pd.read_csv(f'../overviews/overview-msft/{country}_overview.csv')


In [23]:
out_eubucco

{'austria': {'Total Footprint Area': 867271697,
  'Total Footprint Number': 4135733},
 'belgium': {'Total Footprint Area': 1096372592,
  'Total Footprint Number': 8634500},
 'bulgaria': {'Total Footprint Area': 145130155,
  'Total Footprint Number': 448470},
 'croatia': {'Total Footprint Area': 147590430,
  'Total Footprint Number': 873080},
 'cyprus': {'Total Footprint Area': 74417048,
  'Total Footprint Number': 467594},
 'czechia': {'Total Footprint Area': 673376186,
  'Total Footprint Number': 4044659},
 'denmark': {'Total Footprint Area': 738533831,
  'Total Footprint Number': 5691756},
 'estonia': {'Total Footprint Area': 132899346,
  'Total Footprint Number': 803218},
 'finland': {'Total Footprint Area': 691145893,
  'Total Footprint Number': 5370223},
 'france': {'Total Footprint Area': 5851128662,
  'Total Footprint Number': 47847462},
 'germany': {'Total Footprint Area': 6108343563,
  'Total Footprint Number': 43644887},
 'greece': {'Total Footprint Area': 187420816,
  'Total

In [16]:
for country in countries_msft:
    # Sample markdown content
    markdown_content = f"""
# {country.capitalize()}
## Available data EUBUCCO / MSFT

| Dimension    | EUBUCCO v0.1 | MSFT | Ratio |
| -------- | ------- | ------- | ------- |
|   |     |  | |


Total Footprint Area: {out[country]['Total Footprint Area']:,}
Total Footprint Number: {out[country]['Total Footprint Number']:,}

## Statistics
## Maps
## Outliers
## Known issues
"""

    # Write the content to a markdown file
    with open(f"../factsheets/{country}_analysis.md", "w") as file:
        file.write(markdown_content)
